# Vesikel recycling model with different LPA affect times

A simple two pool model of spontaneous vesicle recycling. It is a simplified version of the model published in [Sara et al 2005](https://doi.org/10.1016/j.neuron.2004.12.056).

Here we change the model from Sara 2005 to remove the pool of empty-recycled vesicles.
Instead, recycled vesicles go back into the loaded vesicle pool

We assume three pools:

- u1: Vesicles currently in the resting state
- u2: Vesicles currently activated/merged with the the pre-synaptic membrane
- u3: Vesicles currently being recycled after endocytosis

in addition we have three parameters:

- α: activation/exocytosis rate of vesicles in u1
- β: recycling rate from the membrane back to the resting pool
- σ: vesicle endo-cytosis from the memebrane rate

In [ ]:
begin
    import Pkg
    Pkg.activate(mktempdir())
    Pkg.add("DifferentialEquations")
    Pkg.add("Plots")
    Pkg.add("WebIO")
    Pkg.add("Interact")

	using DifferentialEquations
	using Plots
	using Interact
    using WebIO
    WebIO.install_jupyter_nbextension()
	
	plotly()
end


 Activating new environment at `/tmp/jl_c0pJ9p/Project.toml`
   Updating registry at `~/.julia/registries/General`
┌ Warning: could not download https://juliahub.com./registries
└ @ Pkg.Types /build/julia/src/julia-1.5.4/usr/share/julia/stdlib/v1.5/Pkg/src/Types.jl:951
  Resolving package versions...
Updating `/tmp/jl_c0pJ9p/Project.toml`
  [0c46a032] + DifferentialEquations v6.16.0
Updating `/tmp/jl_c0pJ9p/Manifest.toml`
  [c3fe647b] + AbstractAlgebra v0.16.0
  [1520ce14] + AbstractTrees v0.3.4
  [79e6a3ab] + Adapt v3.3.0
  [ec485272] + ArnoldiMethod v0.1.0
  [4fba245c] + ArrayInterface v3.1.11
  [4c555306] + ArrayLayouts v0.6.5
  [56f22d72] + Artifacts v1.3.0
  [aae01518] + BandedMatrices v0.16.8
  [764a87c0] + BoundaryValueDiffEq v2.7.1
  [fa961155] + CEnum v0.4.1
  [d360d2e6] + ChainRulesCore v0.9.41
  [861a8166] + Combinatorics v1.0.2
  [38540f10] + CommonSolve v0.2.0
  [bbf7d656] + CommonSubexpressions v0.3.0
  [34da2185] + Compat v3.28.0
  [e66e0078] + CompilerSupportLibraries_j

  Resolving package versions...
Updating `/tmp/jl_c0pJ9p/Project.toml`
  [91a5bcdd] + Plots v1.13.2
Updating `/tmp/jl_c0pJ9p/Manifest.toml`
  [6e34b625] + Bzip2_jll v1.0.6+5
  [83423d85] + Cairo_jll v1.16.0+6
  [35d6a980] + ColorSchemes v3.12.1
  [3da002f7] + ColorTypes v0.11.0
  [5ae59095] + Colors v0.12.8
  [d38c429a] + Contour v0.5.7
  [5ae413db] + EarCut_jll v2.1.5+1
  [2e619515] + Expat_jll v2.2.7+6
  [c87230d0] + FFMPEG v0.4.0
  [b22a6f82] + FFMPEG_jll v4.3.1+4
  [53c48c17] + FixedPointNumbers v0.8.4
  [a3f928ae] + Fontconfig_jll v2.13.1+14
  [d7e528f0] + FreeType2_jll v2.10.1+5
  [559328eb] + FriBidi_jll v1.0.5+6
  [0656b61e] + GLFW_jll v3.3.4+0
  [28b8d3ca] + GR v0.57.4
  [d2c73de3] + GR_jll v0.57.2+0
  [5c1252a2] + GeometryBasics v0.3.12
  [78b55507] + Gettext_jll v0.20.1+7
  [7746bdde] + Glib_jll v2.59.0+4
  [42e2da0e] + Grisu v1.0.2
  [cd3eb016] + HTTP v0.9.8
  [83e8ac13] + IniFile v0.5.0
  [c8e1da08] + IterTools v1.3.0
  [682c06a0] + JSON v0.21.1
  [aacddb02] + JpegTurbo_jl

In [ ]:
function vesicle_recycle!(du, u, p, t_span)
    α, β, σ = p.α, p.β, p.σ
    du[1] = -α * u[1] + β * u[3]      # vesicles in the resting pool 
    du[2] = +α * u[1] - σ * u[2]      # vesicles currently merged with the membrane
    du[3] = +σ * u[2] - β * u[3]      # currently being recycled vesicles
end

Setting the initial state of the system:

In [ ]:
u0 = [1.0, 0.0, 0.0];
t_span = (0.0,1200.0);
p = (α=1/120, β=0.5, σ=1.67);

In [ ]:
function first_lpa_time_affect(y, t, integrator)
    return t < 200
end

In [ ]:
function first_para_affect!(integrator)
    integrator.p = (α=1/120, β=0.005, σ=1.67)
end

In [ ]:
function second_lpa_time_affect(y, t, integrator)
    return t < 500
end

In [ ]:
function second_para_affect!(integrator)
    integrator.p = (α=1.5/120, β=0.005, σ=1.67)
end


In [ ]:
callbacks = CallbackSet(
    ContinuousCallback(first_lpa_time_affect, first_para_affect!),
    ContinuousCallback(second_lpa_time_affect, second_para_affect!));

prob = ODEProblem(vesicle_recycle!,u0,t_span,p);
sol = solve(prob, Tsit5(), abstol = 1e-9, reltol = 1e-9, callback=callbacks);

In [ ]:
Plots.theme(:juno)
plot(sol, vars=(0,2))

In [ ]:
@manipulate for x=0:1:10
    x
end
